### RAG를 구성하고 Chroma 벡터 DB를 활용하여 질의하고 응답을 비교해보자!
- Chroma DB 공식 페이지: https://www.trychroma.com/


In [2]:
from openai import OpenAI
from getpass import getpass
import re
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain. text_splitter import RecursiveCharacterTextSplitter

In [3]:
MY_API_KEY = getpass("Enter your OpenAI API key: ")


In [4]:

client=OpenAI(api_key=MY_API_KEY)

## gpt 3.5 모델에게 일반적인 질의

In [5]:
completion=client.chat.completions.create(model='gpt-3.5-turbo', messages=[{'role':'user','content':'기술평가가 뭐야?'}], temperature=0)
                                                                           
print(completion.choices[0].message.content)

기술평가는 특정 기술이나 제품의 성능, 품질, 안전성, 신뢰성 등을 평가하는 과정을 말합니다. 이를 통해 해당 기술이나 제품의 장단점을 파악하고 개선할 수 있는 방향을 제시할 수 있습니다. 주로 연구 및 개발 단계에서 사용되며, 새로운 기술이나 제품을 시장에 출시하기 전에 필수적으로 수행됩니다.


### RAG 구조를 활용한 질의

#### 1)참조 데이터 로드 및 청킹

In [9]:
#한번에 여러개의 pdf 로드
loaders=[PyPDFLoader('기술보증기금과 한국경제.pdf'),  #5page
         PyPDFLoader('기술보증기금 주요업무.pdf')   #4page
]

In [13]:
loaders

In [10]:
#Recursive splitter 객체 생성
pdf_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,
                                            chunk_overlap=50)   

In [11]:
my_chunks=[] #청크를 보관할 리스트

for loader in loaders:
    #2개의 pdf 파일에서 추출된 document 객체들을 하나의 리스트에 합쳐줌
    #extend: 리스트에 다른 리스트의 요소들을 추가
    my_chunks.extend(loader.load_and_split(pdf_splitter))


print('청크 개수:',len(my_chunks))
my_chunks
#두 pdf 파일이 총 page수는 9였지만, chunk_size 를 1000으로 청킹을 진행한 결과 총 13개의 청크가 생성됨

청크 개수: 13


[Document(metadata={'source': '기술보증기금과 한국경제.pdf', 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는 개인 및 법인 \n신용보증 상시 사용하는 종업원이 1천명 이하이고 총자산액이 1천억원 이하인 기업이 부담하는 금전\n채무 보증. \n기술평가 해당 기술과 관련된 기술성·시장성·사업타당성 등을 종합적으로 평가하여 \n금액·등급·의견 또는 점수 등으로 표시하는 것을 말한다 \n* 기금은 기술평가의 객관성 및 공정성 등을 확보하기 위하여 기술평가의 \n기준·절차·방법·종류 등에 관한 사항을 미리 정하여야 한다 \n

### 2) 임베딩 설정

In [12]:
!pip install langchain_chroma


In [13]:
#langchain에서 지원해주는 chroma 클래스 import
from langchain_chroma import Chroma 
#langchain과 연동된 openai에서 지원하는 pre-trained Embedding 모델 활용
from langchain_openai import OpenAIEmbeddings


In [14]:
my_embedding=OpenAIEmbeddings(model='text-embedding-3-small',
                               api_key=MY_API_KEY)


In [20]:
#임베딩 예시
my_list=['오늘 눈이 너무 와서 빨리 집에 가야할것 같아요','빨리 집에 갑시다']

#embed_documents: 임베딩 진(문자열이 들어있는 리스트로 입력, 임베딩 차원은 디폴트 1536차원)
temp_embedding=my_embedding.embed_documents(my_list)
temp_embedding

[[-0.003220065962523222,
  0.017526887357234955,
  -0.08052987605333328,
  0.018742825835943222,
  0.01867334358394146,
  -0.010795798152685165,
  0.020254062488675117,
  -0.01194225437939167,
  -0.07684732228517532,
  -0.03831943869590759,
  -0.03178811073303223,
  0.06513956934213638,
  0.019055495038628578,
  -0.03151018172502518,
  0.013010542839765549,
  -0.04366956651210785,
  -0.01734449528157711,
  -0.002038868609815836,
  0.011004244908690453,
  0.014678115956485271,
  -0.027983959764242172,
  -0.01332321297377348,
  0.008911092765629292,
  -0.004320924635976553,
  0.013748792000114918,
  0.0014298135647550225,
  0.06500060111284256,
  0.01135165523737669,
  0.06451422721147537,
  -0.0529107004404068,
  -0.025691045448184013,
  -0.036825571209192276,
  -0.00031239850795827806,
  -0.056697480380535126,
  0.016198039054870605,
  0.05207691341638565,
  0.03493218123912811,
  0.041967254132032394,
  -0.05409189686179161,
  -0.02817503549158573,
  0.004881124943494797,
  -0.0505830

In [21]:
np.array(temp_embedding).shape
#2개의 문자열이 각각 디폴트 값인 1536차원의 벡터로 임베딩된 것을 확인인

(2, 1536)

### 3) Chroma DB 생성

In [52]:
# DB 생성 경로 설정
my_directory='VectorStores/'

#from_documents: 청킹된 문서(document 객체)의 정보 및 임베딩, 경로정보로 Chroma DB 생성
vectordb=Chroma.from_documents(documents=my_chunks,
                               embedding=my_embedding,
                               persist_directory=my_directory)

### 한번 생성된 DB 불러와서 사용하기
- 프로그램 종료 후 다시 켰을 때 코드를 재실행하게 되면 중첩되어져서 DB에 저장될 것이기 때문에 
- 한번 db를 생성하면 생성하는 코드를 다시 실행시키지말고 생성된 DB를 불러와서 사용할 것.

In [56]:
my_embedding= OpenAIEmbeddings(model='text-embedding-3-small',openai_api_key=MY_API_KEY)
my_directory='VectorStores/'

#기존 생성된 db를 불러올때는 embeding_function에 임베딩 객체 넣기
vectordb=Chroma(persist_directory = my_directory, embedding_function=my_embedding)

#### chroma 클래스로 자주 사용되는 메소드
- _collection.count: 수집한 벡터의 계수 확인
- _collection.get: 수집된 벡터의 정보 확인
- delete: 특정 id를 가진 벡터 삭제
- add_documents: 여러 문서를 벡터 저장소에 추가
- add_texts: 여러 텍스트를 벡터 저장소에 추가
- add_images: 여러 이미지를 벡터 저장소에 추가
- similarity_search: 유사도 계산을 통해 입력 질의에 대해 가장 유사한 문서 검색색

(1) DB 정보 확인

In [26]:
# 수집된 청크개수 확인(여러번 벡터db 생성시 값이 계속 누적되어 저장됨됨)
vectordb._collection.count()

13

In [29]:
#수집된 청크의 정보확인
vectordb._collection.get()

{'ids': ['a914243a-4702-4950-ac87-55bdf56850d4',
  '25517c3d-913e-4a2f-927d-c6f9f0b48482',
  '6fd1cadd-d97d-4961-b703-07e7c6b75ac8',
  '2e8b4e37-f90e-4833-87b0-ceb37b102b7f',
  '24926ed0-ba43-49e4-bdfc-1870a67123da',
  '7a80ce28-b8b2-4d32-ae0d-3535bd57403e',
  '7c4c213c-bc16-4a64-af2b-3cd69d70665a',
  '9c0416db-c400-4249-a027-1dfa47120b4b',
  '7af4bd39-f7ea-414b-816c-6bae7af70c59',
  '2d331ef5-7849-4f90-9e04-a99bd22585bb',
  'e7a4fe41-f062-45df-ab4b-da442b853582',
  '0a46b97a-a66f-453b-80ec-61ae4f2e635d',
  'b9c21390-7ea8-478e-ae62-e34841b72cdd'],
 'embeddings': None,
 'documents': ['페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인

(2) DB에 값 추가

In [31]:
vectordb.add_texts(texts=['sample_text1', 'sample_text2'])
#추가된 텍스트에 id가 부여되어 표시됨

['07ebb7a6-f0f4-4a0a-9e8d-b0e3d9e24c6d',
 'd752121f-26c3-422b-9f09-68ddc87b1c9b']

In [36]:
vectordb._collection.get()['ids']

['a914243a-4702-4950-ac87-55bdf56850d4',
 '25517c3d-913e-4a2f-927d-c6f9f0b48482',
 '6fd1cadd-d97d-4961-b703-07e7c6b75ac8',
 '2e8b4e37-f90e-4833-87b0-ceb37b102b7f',
 '24926ed0-ba43-49e4-bdfc-1870a67123da',
 '7a80ce28-b8b2-4d32-ae0d-3535bd57403e',
 '7c4c213c-bc16-4a64-af2b-3cd69d70665a',
 '9c0416db-c400-4249-a027-1dfa47120b4b',
 '7af4bd39-f7ea-414b-816c-6bae7af70c59',
 '2d331ef5-7849-4f90-9e04-a99bd22585bb',
 'e7a4fe41-f062-45df-ab4b-da442b853582',
 '0a46b97a-a66f-453b-80ec-61ae4f2e635d',
 'b9c21390-7ea8-478e-ae62-e34841b72cdd',
 '07ebb7a6-f0f4-4a0a-9e8d-b0e3d9e24c6d',
 'd752121f-26c3-422b-9f09-68ddc87b1c9b']

- db에 문서 추가하기


In [40]:
#새롭게 추가할 문서 2개를 로드 후 청킹
add_loaders=[PyPDFLoader('기술보증기금 주요업무_기술보증.pdf'), 
            PyPDFLoader('기술보증기금 주요업무_기술평가.pdf')
            ]
add_chunks=[]
for loader in add_loaders:
    add_chunks.extend(loader.load_and_split(pdf_splitter))
print(len(add_chunks))

15


In [41]:
#벡터 DB에 문서 추가 명령 실행
vectordb.add_documents(documents=add_chunks)

['12d2d171-114b-4b67-92b9-40d261ce3cff',
 'd81a05d2-463f-4879-9268-3f722451aaa5',
 'e1fc5c7e-26fa-4b87-b68e-a4360048d0cf',
 'dc46c0e4-c3cb-469c-a891-55fc6aa702dc',
 'a37d5063-ea60-44c4-b4ec-f4c6342887ba',
 'dc289579-942e-4714-aca6-b6fb8c7b1ca3',
 '089eaf98-0404-4ba3-97cb-ed250aa14225',
 '597c7d84-f2a8-4b53-9bc3-433b8e34da47',
 '109f80ce-88c2-465f-88cd-1db6b403cca3',
 'a8cc5be8-7cd6-40d3-8615-cd155341474a',
 '650df450-97dd-42e0-8509-3bfacd27a56a',
 '5425e463-1238-434c-981b-95191e389ee5',
 '8ca3adf3-dd8c-40d8-9c64-b4a442d182c9',
 '16409c6a-6c08-4f7a-9f2d-91fafe3f8d94',
 'f9ade5eb-2ab5-48f0-9357-f7880a0e0cda']

In [42]:
vectordb._collection.count()
#기존 13개에서 text로 2개 추가하고, 문서로 15개 추가되어서 30개

30

(3) DB값 삭제

In [44]:

#text로 추가한 값 2개 삭제
vectordb.delete(ids=['07ebb7a6-f0f4-4a0a-9e8d-b0e3d9e24c6d',
 'd752121f-26c3-422b-9f09-68ddc87b1c9b'])
vectordb._collection.count()

28

In [ ]:
#모든 데이터 삭제
vectordb.delete_collection()
vectordb._collection.count()

ValueError: Chroma collection not initialized. Use `reset_collection` to re-create and initialize the collection. 